## Methodology

A modified CycleGAN was used in order to de-noise the data without paired data. CycleGANs operate by having two GANs, with the output of one being fed into the other to verify the image does not structurally change throughout the process. 

Since RES data can be variable in size, it was necessary to split the data into smaller chunks so it could be fit into the network. Then, after processing, the data could be stitched back together. 

For normalization, all data were scaled bewteen 0 and 255

In order to train the network, a domain of clean data and noisy data were needed. Data from the HiCARS radar was used for the domain of noisy data, however locating a domain of data in which the noise was completely removed yet the structure of the data remained proved difficult. In order to obtain the clean domain, it was necessary to simulate clean data using the Bruges library. The code to simulate the data randomly is as follows:

In [4]:
import numpy as np
import bruges as bg
from PIL import Image
import random

def contains_all_nums(strata, max_layers):
    for i in range(0, max_layers):
        if i not in strata:
            return False
    return True

def calc_split_points(strata):
    split_points = np.array([random.random() for _ in range(2)])
    split_points *= random.randint(8, len(strata) - 2)/np.amax(split_points)
    
    split_points = np.round(split_points)
    split_points = np.sort(split_points)
    return split_points

def make_random_wedges():
    depth = (random.randint(600,1200), random.randint(800, 2000), random.randint(400,1200))
    width = (random.randint(600,1200), random.randint(800, 2000), random.randint(400,1200))
    mode = random.choice(['linear', 'clinoform'])
    conformance = random.choice(['top', 'bottom', 'proportional'])
    max_layers = random.randint(4, 10)
    
    strata = [random.randint(0, max_layers) for i in range(5, 20)]
    while not contains_all_nums(strata, max_layers):
        strata = [random.randint(0, max_layers) for i in range(5, 20)]
    
    threshold = random.randint(2, max(len(strata)//4,2))
    split_points = calc_split_points(strata).astype('int8')
    while split_points[1] - split_points[0] < threshold:
            split_points = calc_split_ŒŒpoints(strata).astype('int8')
    upper_strata = tuple(strata[:split_points[0]])
    middle_strata = tuple(strata[split_points[0]:split_points[1]])
    lower_strata = tuple(strata[split_points[1]:])
    starting_thickness = random.uniform(0, 2)
    thickness=(starting_thickness, starting_thickness + random.uniform(0,2))
    w, top, base, ref = bg.models.wedge(depth=depth,
                                    width=width,
                                    strat=tuple([upper_strata, middle_strata, lower_strata]),
                                    mode=mode,
                                    thickness=thickness,
                                   )
    return w
done = False
while not done:
    try:
        w = make_random_wedges()
        rocks = np.array([i+1 for i in (np.unique(w))])
        # Fancy indexing into the rocks with the model.
        impedance = rocks[w-1]
        # Make reflectivity.
        rc = (impedance[1:] - impedance[:-1]) / (impedance[1:] + impedance[:-1])
        rc -= np.amin(rc)
        rc *= 255/(np.amax(rc)-np.amin(rc))


        impedance -= np.amin(impedance)
        impedance = impedance.astype('float64')
        impedance *= 255/(np.amax(impedance) - np.amin(impedance))
        impedance = np.round(impedance).astype('int8')
        Image.fromarray(impedance).convert("RGB").convert("L").show()
        done=True
    except:
        pass

The generated data looks similar to the following.

![image](resources/data.png)

Once the clean domain of data was generated, the data were prepared for use in the network. Since the images were of variable dimensions, the network was designed to accept chunks of constant size, so as to clean each piece of data in chunks. The code to split the data into chunks is as follows:

In [5]:
def split_data(data, chunk_size):
    num_vertical = data.shape[0]//chunk_size
    if data.shape[0] % chunk_size != 0:
        num_vertical += 1
    num_horizontal = data.shape[1]//chunk_size
    if data.shape[1] % chunk_size != 0:
        num_horizontal += 1
    
    chunks = []
    
    for i in range(num_vertical):
        for j in range(num_horizontal):
            if i == num_vertical - 1 or j == num_horizontal - 1:
                if i == num_vertical - 1 and j == num_horizontal - 1:
                    tmp = data[i*chunk_size:,j*chunk_size]
                elif i == num_vertical -1:
                    tmp = data[i*chunk_size:,j*chunk_size:(j+1)*chunk_size]
                else:
                    tmp = data[i*chunk_size:(i+1)*chunk_size,j*chunk_size:(j+1)*chunk_size]
                if tmp.ndim == 1:
                    tmp = tmp.reshape(-1,1)
                chunks.append(tmp)
            else:
                chunks.append(data[i*chunk_size:(i+1)*chunk_size,j*chunk_size:(j+1)*chunk_size])
    return chunks

Then, with the data chunked – and padded as needed – the data is fed into the CycleGAN for training. In order to measure the loss, the standard CycleGAN losses were used: an adversarial loss to train the generators and an identity loss to ensure consistency between input and output data. However, since the simulated data is somewhat structurally different from the HiCARS data, a hybrid training approach was used. In the intial training epochs, the network was trained on images filtered using traditional techniques, then the training data was alternated to the simulated data in order to attempt to reduce the noise while maintaining the structure of the data. 

Then, to track the accuracy in de-noising the images, a measurement of the Peak-Signal-To-Noise-Ratio (PSNR) was used. To calculate this, simulated data is passed backward through the network in order to add noise to the image, then the image is pased backward again and the amount of remaining noise is used to measure the accuracy, since the exact signal is known. Given a piece of data D1 with dimensions m x n, D1(i,j) refers to the value at the pixel at (i,j). Given added noise N with dimensions m x n, the noisy data can be calculated as D2 = D1 + N. Then, the PSNR can be calculated by:

![image](resources/PSNR.png)